In [1]:
from qutip import *

In [ ]:
# As = A simulation
Is = identity(2)
Xs = sigmax()
Ys = sigmay()
Zs = sigmaz()

# Hamiltonians
H1s = (w1/2)*Zs
H2s = (w2/2)*Zs

Hcs = (wc/2)*Zs
Hhs = (wh/2)*Zs

In [ ]:
#States
q0 = Qobj([[1],[0]])
q1 = Qobj([[0],[1]])

rho0 = fock_dm(2,0)

#States
psi1 = q1
psi2 = q1
rho1 = ket2dm(psi1)
rho2 = ket2dm(psi2)

#Reservoirs
rhoc = (-beta_c*Hcs).expm()/((-beta_c*Hcs).expm().tr())
rhoh = (-beta_h*Hhs).expm()/((-beta_h*Hhs).expm().tr())

In [ ]:
sigp = (Xs + 1j*Ys)/2;
sigm = (Xs - 1j*Ys)/2;

Vc = g12 * (tensor(sigp,sigm,Is,Is) + tensor(sigm,sigp,Is,Is))
Vh = g12 * (tensor(Is,Is,sigp,sigm) + tensor(Is,Is,sigm,sigp))
Vs = g12 * (tensor(Is,sigp,sigm,Is) + tensor(Is,sigm,sigp,Is))

#Hamiltonians
Hqs = (tensor(Hcs,Is,Is,Is) + tensor(Is,Is,Is,Hhs) + 
       tensor(Is,H1s,Is,Is) + tensor(Is,Is,H2s,Is) +
       Vc + Vh)
Hws = (tensor(Is,H1s,Is,Is) + tensor(Is,Is,H2s,Is) + 
      Vs)

Uqs = (-1j*Hqs*tq).expm()
Uws = (-1j*Hws*tw).expm()

In [ ]:
#System state
rhows = tensor(rhoc,rho1,rho2,rhoh)

Qc_sim = []
Qh_sim = []
W_sim = []

for i in range(N):
    #Heat Stroke
    rhoqs = Uqs*rhows*(Uqs.dag())
    
    Qc_sim.append(expect(tensor(Is,H1s,Is,Is),rhoqs-rhows))
    Qh_sim.append(expect(tensor(Is,Is,H2s,Is),rhoqs-rhows))
    
    #Work Stroke
    rhows = Uws*rhoqs*(Uws.dag())
    
    W_sim.append(expect(-tensor(Is,H1s,Is,Is)-tensor(Is,Is,H2s,Is),rhows-rhoqs))
    
    #Resetting the ancillas 
    rhoaux = ptrace(rhows,[1,2])
    rhows = tensor(rhoc,rhoaux,rhoh)

In [ ]:
print("Classical simulation is finished.")